In [1]:
## Importieren der erforderlichen Bibliotheken
# Pyhton Version 3.11.1
# Selenium Version 4.9.1
# webdriver_manager Version 3.8.6
# pandas Version  2.0.1
# datetime Version 5.1
# sqlalchemy Version 2.0.13
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random
import pandas as pd
import datetime
from sqlalchemy import create_engine
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText



## Hilfsfunktionen 
# Log File
def schreibe_log_file(log_eintrag):
    datei='log-Stepstone.txt' # Name der Log-Datei
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Überprüfen, ob die Log-Datei bereits existiert
    if not os.path.exists(datei):
        # Wenn die Log-Datei nicht existiert, wird sie erstellt und der Log-Eintrag wird hineingeschrieben
        with open(datei, 'w') as file:
            file.write(f"{timestamp}, {log_eintrag}" + '\n')
    else:
        # Wenn die Log-Datei bereits existiert, wird der Log-Eintrag an das Ende der Datei angehängt
        with open(datei, 'a') as file:
            file.write(f"{timestamp}, {log_eintrag}" + '\n')
            
# E-Mail - Benachrichtigung bei Fehlern
def schreibe_e_mail(message, Subject="fehlerhaft"):
    # SMTP-Server-Konfiguration
    host = "smtp.web.de"
    port = 587
    log_in_id = ""
    passwort = ""

    # Erstellen der E-Mail-Nachricht
    msg = MIMEMultipart()
    msg['From'] = log_in_id + "@web.de" # Absender
    msg['To'] = "jobs@data-craft.de" # Empfänger
    msg['Subject'] = f'Stepstone Abfrage {Subject}' # Betreff der E-Mail

    msg.attach(MIMEText(message, 'plain')) # Hinzufügen des Nachrichtentextes zur E-Mail

    # Verbindung zum SMTP-Server herstellen und E-Mail senden
    with smtplib.SMTP(host=host, port=port) as mail:
        mail.starttls() # Starte TLS-Verschlüsselung
        mail.login(log_in_id, passwort) # Anmeldung am SMTP-Server
        mail.send_message(msg) # E-Mail senden

# random Wartezeit zum für seitenaufbau oder so
def wartezeit(zeit=3):
    """
    Diese Funktion fügt eine zufällige Wartezeit hinzu, bevor sie fortgesetzt wird.

    :param zeit: Die maximale Wartezeit in Sekunden (Standardwert ist 3).
    :return: None
    """
    time.sleep(random.randint(1, zeit))


## SQL-Server verbindung erstellen
# SQL-Server-Konfiguration
host=""
database=""
user=""
password=""
tabelle_Rohdaten="jobs.rohdaten"

# Erstelle eine SQL-Verbindung mit der Datenbank
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}')



## Der Scraper
def scraper(jobtitel, suchort='Deutschland'):
    '''
    Der eigentliche Scraper für die Seite
    
    :param jobtitel: Der Titel des Jobs, nach dem gesucht werden soll.
    :param suchort: Der Ort, an dem nach Jobs gesucht werden soll. Standardwert ist 'Deutschland'.
    '''
    
    schreibe_log_file(f'suche nach {jobtitel}')
    
    # ChromeOptions erstellen
    chrome_options = webdriver.ChromeOptions()
    #Headless-Modus aktivieren
    #chrome_options.add_argument("--headless")

    # Browserfenster öffnen nach Einstellung in den Optionen
    try:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
        wartezeit(1) # Wartezeit das Browser geladen ist
        schreibe_log_file('Browser geoeffnet')
    except:
        schreibe_log_file('Browser konnte nicht geoeffnet werden')
        schreibe_e_mail(f'Browser konnte nicht geoeffnet werden: -- {datetime.datetime.now().strftime("%d.%m.%Y, %H:%M:%S")} --')

    # Webseite aufrufen
    driver.get("https://www.stepstone.de/")

    # Fenster Maximieren
    driver.maximize_window() # evtl. im Headless-Modus nicht machbar

    # Cookies akzeptieren
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="ccmgt_explicit_accept"]').click()
    wartezeit(1)
    
    ## Suchfelder finden, anwählen und befüllen
    # Suchfeld Jobbezeichnung -> finden und befüllen
    suchfeld_jobtitel=driver.find_element(By.XPATH,'//*[@id="stepstone-autocomplete-162"]')
    suchfeld_jobtitel.send_keys(jobtitel)
    wartezeit(1)

    # Suchfeld Ort -> finden und befüllen
    suchfeld_ort=driver.find_element(By.XPATH,'//*[@id="stepstone-form-element-173-input"]')
    suchfeld_ort.send_keys(suchort)
    wartezeit(1)
    
    # Anfrage mit Enter/Return abschicken
    suchfeld_ort.send_keys(Keys.RETURN)
    wartezeit(1)
    
    ## nach Datum sortieren
    # erst Filterfeld dann Datum klicken
    driver.find_element(By.CLASS_NAME, 'res-1vztcyh').click()
    wartezeit(1)
    driver.find_element(By.XPATH, '//*[@id="date"]/span/span[2]').click()
    time.sleep(3)

    # Leere Liste erstellen für die Links
    link_liste_scraped=[]
    
    ## Namen für verschiedene XPATH, CLASS, CSS, etc.
    # name für den Container der alle Stellenanzeigen enthält
    class_name_anzeige_link='res-kyg8or'

    # name für den Container für alle URLs der einzelnen Stellenanzeigen
    # da es meherer Namen gibt, mit Schleife herausfinde welche es gibt
    for class_name in ['res-3yv1ty','res-2cltag']:
        if driver.find_elements(By.CLASS_NAME, class_name) != []:
            class_name_url_link=class_name
    
    ## Aus den Stellenanzeigen die Links zu den Stellenanzeigen finden und in eine Liste packen
    # alle Stellenanzeigen auf der seite finden
    anzeigen = driver.find_elements(By.CLASS_NAME, class_name_anzeige_link)

    # in jeder Stellenazige die URL heraussuchen
    for anzeige in anzeigen:
        try:
            for link in anzeige.find_elements(By.CLASS_NAME, class_name_url_link):
                link_liste_scraped.append(link.get_attribute('href'))
        except:
            schreibe_log_file(f'Link in Anzeige auf erster Seite nicht gefunden')
            continue
            
    # weitere Seiten aufrufen
    for i in range(9):
        
        # Link zur nächsten seite finden und aufrufen
        try:
            driver.get(driver.find_elements(By.CLASS_NAME, 'res-1w7ajks')[1].get_attribute('href'))
        except:
            schreibe_log_file('Class-Name nicht gefunden für nächsten seite der Stellenanzeigen')
            pass
        #Warten auf den Aufbau der Seite
        time.sleep(2)
        
        # alle Stellenanzeigen auf der seite finden
        anzeigen = driver.find_elements(By.CLASS_NAME, class_name_anzeige_link)

        # in jeder Stellenazige die URL heraussuchen
        for anzeige in anzeigen:
            for link in anzeige.find_elements(By.CLASS_NAME, class_name_url_link):
                link_liste_scraped.append(link.get_attribute('href'))
                
    # überprüfen ob link_liste_scraped leer ist da evtl. die Class name geändert wurde
    if len(link_liste_scraped) == 0:
        schreibe_log_file(f'Keine Links auf gefunden: Jobtitel => {jobtitel}')
        schreibe_e_mail(f'Keine Links auf Webseite gefunden: Jobtitel => {jobtitel}\n\n Hinweis: Classenname überprüfen für alle Stellenanzeigen!')
        return
    else:
        pass
    
    # Duplikate entfernen
    link_liste_scraped_ohne_duplikate = list(set(link_liste_scraped))
    schreibe_log_file(f'Es wurden {len(link_liste_scraped)-len(link_liste_scraped_ohne_duplikate)} Duplikate in der "Link Liste" entfernt')
    schreibe_log_file("link liste erstellt")
    
    # Lese die bestehenden URLs aus der Datenbank
    existing_urls = pd.read_sql_query(f"SELECT url FROM {tabelle_Rohdaten}", connection)
    schreibe_log_file(f'{len(existing_urls)} in der datenbank bereits vorhanden')

    # Filtere den DataFrame, um nur neue URLs zu behalten
    link_liste = [url for url in link_liste_scraped_ohne_duplikate if url not in existing_urls['url'].values]
    
    # Listen für für den DataFrame
    seiten_inhalt_html_liste = []
    seiten_inhalt_liste = []
    URL_Liste = []
    Datum_Liste = []

    ## Scrapen
    try:
        schreibe_log_file('Abfrage begonnen')
        for link in link_liste:
            
            # Aufrufen der Seite
            try:
                driver.get(link)
                wartezeit(2)
            except:
                schreibe_log_file(f'Fehler beim aufrufen der URL: {link}')
            
            # Prüfen ob ein bestimmter Text auf der Seite steht, der auf die nicht mehr existierende Anzeige hindeutet
            try:
                stellenanzeige_pruefen=driver.find_element(By.CLASS_NAME, 'listing-content-provider-1qtvd67').text
            except:
                stellenanzeige_pruefen=""
            
            # Prüfen ob Stellenanzeige existiert oder nicht 
            if stellenanzeige_pruefen != 'Diese Stellenanzeige ist nicht mehr verfügbar.':
                            
                ## Grunddaten scrapen
                seiten_inhalt_html_liste.append(driver.find_element(By.CLASS_NAME, 'reb-main').get_attribute('innerHTML'))
                seiten_inhalt_liste.append(driver.find_element(By.CLASS_NAME, 'reb-main').text)
                    
                # URL der Stellenanzeige
                URL_Liste.append(link)
                
                # Zeitstempel
                Datum_Liste.append(datetime.datetime.now())
                
                schreibe_log_file(f'Daten wurden gezogen: {link}')
            else:
                schreibe_log_file(f'Stellenanzeige nicht verfügbar:  {link}')
    except:
        schreibe_log_file("Abfrage abgebrochen !!!")
        schreibe_e_mail(f"Die Abfrage wurde abgebrochen. -- {datetime.datetime.now()} --")
    schreibe_log_file('Abfrage beendet')
    
    # Schließen des Browsers
    driver.quit()
    schreibe_log_file('Browser geschlossen')
    time.sleep(3)
        
    # Prüfen ob die Listen gleich lang sind
    if len(set(map(len, [seiten_inhalt_html_liste,
                        seiten_inhalt_liste,
                        URL_Liste,
                        Datum_Liste]))) > 1:
        schreibe_log_file('Listen sind verschieden lang')
        schreibe_e_mail(f'Listen sind nicht gleich lang  -- {datetime.datetime.now()} --')
        return
    else:
        # DataFrame erstellen
        df = pd.DataFrame({"seite": "stepstone",
                        "seiten_inhalt_html": seiten_inhalt_html_liste,
                        "seiten_inhalt": seiten_inhalt_liste, 
                        "url": URL_Liste, 
                        "datum":Datum_Liste,
                        "storno": False})
        
    ## Daten in die Datenbank einfügen
    # Schreibe den bereinigten DataFrame in die Datenbank
    try:
        df.to_sql(name=tabelle_Rohdaten.split('.')[1],
                schema=tabelle_Rohdaten.split('.')[0],
                con=connection, if_exists='append', index=False, )
        schreibe_log_file(f'Es wurden {len(df)} Daten von Stepstone hinzugefügt')
    except:
        schreibe_log_file('Datenbank konnte nicht gefüllt werden => Dataframe fehlt')
        schreibe_e_mail(f'Datenbank konnte nicht gefüllt werden => Dataframe fehlt  -- {datetime.datetime.now()} --')

    # Verbindung zur SQL-Datenbank schließen
    connection.dispose()
print("fertig")

fertig


In [2]:
# Ich arbeite mit zwei Computern, und leider hat die Datei für die Suchbegriffe (Jobtitel) jeweils ein anderen Speicherort
try:
    suchbegriffe = pd.read_excel(r"C:\Daten@DS-Server\Python\Block 9.2 - Praxisprojekt\Jobtitel.xlsx",
                                sheet_name='Stepstone')
except:
    suchbegriffe = pd.read_excel(r"C:\Daten@Server\Python\Block 9.2 - Praxisprojekt\Jobtitel.xlsx",
                                sheet_name='Stepstone')  
display(suchbegriffe)

,Jobtitel
0,Data Analyst
1,Junior Data Analyst
2,Data Scientist & Machine Learning Engineer
3,Data Expert
4,Data Warehouse Developer
5,(Junior) Data Analyst
6,Data Scientist
7,Customer Data Analyst / Engineer
8,Big Data Engineer
9,Data Consultant


In [3]:
print(f'{len(suchbegriffe)} Jobtitel gefunden')
for i, jobtitel in enumerate(suchbegriffe.Jobtitel):
    print(f'{i}: Suche nach "{jobtitel}", Start: {datetime.datetime.now().strftime("%H:%M:%S")} Uhr')
    scraper(jobtitel)
print("")
print("fertig")

0: Suche nach "Data Analyst", Start: 06:06:17 Uhr


[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 6.61MB/s]


1: Suche nach "Junior Data Analyst", Start: 06:08:15 Uhr
2: Suche nach "Data Scientist & Machine Learning Engineer", Start: 06:10:25 Uhr
3: Suche nach "Data Expert ", Start: 06:11:30 Uhr
4: Suche nach "Data Warehouse Developer", Start: 06:14:26 Uhr
5: Suche nach "(Junior) Data Analyst", Start: 06:16:00 Uhr
6: Suche nach "Data Scientist", Start: 06:17:17 Uhr
7: Suche nach "Customer Data Analyst / Engineer", Start: 06:18:45 Uhr
8: Suche nach "Big Data Engineer", Start: 06:20:24 Uhr
9: Suche nach "Data Consultant", Start: 06:21:43 Uhr
10: Suche nach "Master Data Coordinator", Start: 06:23:47 Uhr
11: Suche nach "Specialist Data Analytics & Master Data", Start: 06:35:48 Uhr
12: Suche nach "Data Enthusiast", Start: 06:48:38 Uhr

fertig
